In [33]:
from langchain_text_splitters import RecursiveCharacterTextSplitter # Essa lib serve para dividir o texto em caracteres de forma recursiva (ou seja, dividir cada caractere em um array de caracteres)
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

from dotenv import load_dotenv


In [9]:
import os
load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [ ]:
embeddings = OpenAIEmbeddings(model='text-embeddings-3-small') # Essa lib serve para pegar os embeddings de um texto (ou seja, pegar a representação vetorial de um texto)
llm = ChatOpenAI(model='gpt-3.5-turbo', max_tokens=500)

# res = llm.invoke('Qual a historia da lei do chá')
# print(res)

content='A história da lei do chá remonta ao século XIX, durante o período colonial britânico na Índia. Em 1835, a Companhia Britânica das Índias Orientais estabeleceu o monopólio do comércio de chá na Índia, o que resultou em um aumento significativo nos preços do chá para os consumidores indianos.\n\nEm resposta a essa situação, o ativista e líder político indiano, Bahadur Shah Zafar, liderou uma campanha para boicotar o chá britânico e promover o consumo de chá indiano. Ele defendeu a produção e o consumo de chá indiano como um ato de resistência contra a exploração britânica.\n\nEm 1837, Bahadur Shah Zafar e seus seguidores organizaram um protesto massivo em Delhi, onde queimaram grandes quantidades de chá britânico como forma de protesto. Esse evento ficou conhecido como a Lei do Chá, e marcou o início de um movimento de resistência contra o domínio britânico na Índia.\n\nA Lei do Chá tornou-se um símbolo importante na luta pela independência da Índia, e inspirou outros líderes in

In [15]:
# Carregar o PDF

pdf_link = 'pdf-investimentos-RAG.pdf'
loader = PyPDFLoader(pdf_link, extract_images=False)

pages = loader.load_and_split()

In [16]:
len(pages)

25

In [19]:
# Splitters

child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

parent_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 4000, # Tamanho do chunk
    chunk_overlap=200, # Sobreposição entre os chunks (para não perder informação / contexto)
    length_function=len, # Função que retorna o tamanho do texto
    add_start_index=True # Adicionar o índice de início do chunk (para saber onde o chunk começa)
)

In [17]:
# Storages
store = InMemoryStore()
vector_store = Chroma(embedding_function=embeddings, persist_directory='child_vectorDB')

/var/folders/3d/sw03jzns4xz8x8zxh2ksmfhw0000gn/T/ipykernel_12345/248056858.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(embedding_function=embeddings, persist_directory='child_vectorDB')


In [ ]:
# Retriever
parent_document_retriever = ParentDocumentRetriever(
    vectorstore=vector_store, # Onde os vetores serão armazenados
    docstore=store, # Onde os documentos serão armazenados 
    child_splitter=child_splitter, # Como dividir o texto em pedaços menores
    parent_splitter=parent_splitter # Como dividir o texto em pedaços maiores
)
print(parent_document_retriever)
parent_document_retriever.add_documents(pages, ids=None) # Os ids são identificadores únicos para cada documento. Se não forem fornecidos, serão gerados automaticamente.
print(parent_document_retriever)

vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x14eae12d0> docstore=<langchain_core.stores.InMemoryStore object at 0x14ecf4210> search_kwargs={} child_splitter=<langchain_text_splitters.character.RecursiveCharacterTextSplitter object at 0x1600a08d0> parent_splitter=<langchain_text_splitters.character.RecursiveCharacterTextSplitter object at 0x1611f6550>
vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x14eae12d0> docstore=<langchain_core.stores.InMemoryStore object at 0x14ecf4210> search_kwargs={} child_splitter=<langchain_text_splitters.character.RecursiveCharacterTextSplitter object at 0x1600a08d0> parent_splitter=<langchain_text_splitters.character.RecursiveCharacterTextSplitter object at 0x1611f6550>


In [24]:
# visualização 
parent_document_retriever.vectorstore.get()

{'ids': ['fca75fd3-e67e-4745-ab1b-f5716048085e',
  '31c1b18e-dd14-4df9-ba1a-aceac19e1ff7',
  '0c46ce2c-937a-480e-ac29-f6b999e54799',
  'aecc8705-6d2c-4b9c-a383-b33fe1c3b604',
  '63d94426-1c39-44d3-82e6-81ee886ccaa7',
  '7c174797-e681-4c1b-b48b-af2dd057c966',
  '7ecd5f42-6145-437c-ae6d-0f4db91938cf',
  '5af8a47a-9b19-4ee4-8cab-ad761b4f4e40',
  'bfb25f42-abb0-4163-b3b9-7aafd8a7f4c7',
  'e17c5cd4-e570-4024-8c83-268ce657a0fe',
  '70eff472-8878-4449-ba7b-ebb482f7a341',
  '4854c97a-61ba-4186-83e5-23506d130382',
  '6585b60e-f125-4bd5-9ac8-8567a1a17824',
  '9d7c3965-06c8-4ff2-af1c-0833914b9ae3',
  '9d210d14-cae3-43c7-b1ba-9e02d2cc8625',
  '3b6aa1ef-6a3e-45f2-bdae-da8203ff23f7',
  '7ec44cbe-b521-4e92-9c74-708898c73abe',
  '527cad8e-3704-4fe3-8ba2-720b2d9f9a56',
  '8b90a1cb-34e2-49af-8b47-1ae5d761f13d',
  '1880ab19-a101-4dbd-8329-ca85af9c337a',
  '53f39dd7-1508-44a2-a225-ff96687a011f',
  'b111c841-9e49-4690-a503-8d26a2417a7f',
  'ccabc1b4-c971-40de-85a5-c630840fb61b',
  'a4115751-cf60-464d-b7d8-

In [32]:
TEMPLATE =  """"
    Você é um especialista em investimentos e temas do tal. Responda a perguntas utilizando o contexto informado.
    Query: 
    {question}
    
    Context:
    {context}
"""

rag_prompt = ChatPromptTemplate.from_template(TEMPLATE)

In [34]:
# Setup do RAG
setup_retrival = RunnableParallel({'question': RunnablePassthrough(), 'context': parent_document_retriever})

output_parser = StrOutputParser()

In [35]:
parent_chain_retrival = setup_retrival | rag_prompt | llm | output_parser

In [38]:
parent_chain_retrival.invoke('faça um binary search')

'Desculpe, mas não posso fazer um binary search com base no contexto fornecido sobre investimentos. O binary search é um algoritmo de busca em que os elementos de uma lista são organizados em ordem crescente ou decrescente e, em seguida, o algoritmo divide repetidamente a lista ao meio e verifica se o elemento desejado está na metade correta. Para realizar um binary search, preciso de uma lista ordenada de elementos para buscar um valor específico. Se você tiver alguma outra pergunta relacionada a investimentos, estou à disposição para ajudar.'